In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
from pathlib import Path
import spikeinterface.full as si
from probeinterface import read_spikeglx
import numpy as np

In [2]:
base_folder = Path('/home/arthur/Documents/SpikeSorting/Test_20210518/') 

data_folder = base_folder / 'raw_awake'

out_path = Path('/media/storage/spikesorting_output/REST/')

recording = si.SpikeGLXRecordingExtractor(data_folder)
print(recording)

probe = read_spikeglx(data_folder / 'raw_awake_01_g0_t0.imec0.ap.meta')
print(probe)
recording = recording.set_probe(probe)
print(recording)

SpikeGLXRecordingExtractor: 385 channels - 1 segments - 30.0kHz - 3881.358s
384
Probe - 384ch
ChannelSliceRecording: 384 channels - 1 segments - 30.0kHz - 3881.358s


In [3]:
fs = recording.get_sampling_frequency()
rest_window = (2000, 2500) # Time in seconds
rest_rec = recording.frame_slice(rest_window[0]*fs, rest_window[1]*fs)
rest_rec = rest_rec.set_probe(probe)
print(rest_rec)

FrameSliceRecording: 384 channels - 1 segments - 30.0kHz - 500.000s


In [5]:
ids = rest_rec.channel_ids[0:11]
w_ts = si.plot_timeseries(rest_rec, time_range=(0,50), channel_ids=ids) # plotting just the first 10 channels
w_ts.figure.suptitle('Recording First 10 Channels')
w_ts.ax.set_ylabel('Channels IDs')

ticks = list(np.linspace(int(w_ts.ax.get_ylim()[0]), int(w_ts.ax.get_ylim()[1]), len(ids)+2))
w_ts.ax.set_yticks(ticks[1:-1])
w_ts.ax.set_yticklabels(ids)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[Text(0, 0.0, 'imec0.ap#AP0'),
 Text(0, 123.0, 'imec0.ap#AP1'),
 Text(0, 246.0, 'imec0.ap#AP2'),
 Text(0, 369.0, 'imec0.ap#AP3'),
 Text(0, 492.0, 'imec0.ap#AP4'),
 Text(0, 615.0, 'imec0.ap#AP5'),
 Text(0, 738.0, 'imec0.ap#AP6'),
 Text(0, 861.0, 'imec0.ap#AP7'),
 Text(0, 984.0, 'imec0.ap#AP8'),
 Text(0, 1107.0, 'imec0.ap#AP9'),
 Text(0, 1230.0, 'imec0.ap#AP10')]

In [6]:
w_probe = si.plot_probe_map(rest_rec, channel_ids=rest_rec.channel_ids[0:4]) # You can change which channels are plotted
w_probe.ax.set_ylim(-100,50)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(-100.0, 50.0)

In [7]:
out_path = Path('/media/storage/spikesorting_output/REST/')
save_folder = out_path / 'save_sorting'
TDC = si.NpzSortingExtractor(save_folder / 'TDCS3_new.npz')
SC = si.NpzSortingExtractor(save_folder / 'SC_new.npz')

In [8]:
rest_rec_bp = si.bandpass_filter(rest_rec, freq_min=300, freq_max=6000) # apply filter
folder_TDC = out_path / 'Phy/TDC-REST'
folder_TDC_wv = folder_TDC / 'waveforms_filtered_new'
we_TDC = si.extract_waveforms(rest_rec_bp, TDC, folder_TDC_wv,
    load_if_exists=True, ms_before=1, ms_after=2., max_spikes_per_unit=500,
    chunk_size=30000, n_jobs=6, progress_bar=True)
print(we_TDC)

WaveformExtractor: 384 channels - 15 units - 1 segments
  before:30 after60 n_per_units: 500


In [9]:
folder_SC = out_path / 'Phy/SC-REST'
folder_SC_wv = folder_SC / 'waveforms_filtered_new'
we_SC = si.extract_waveforms(rest_rec_bp, SC, folder_SC_wv,
    load_if_exists=True, ms_before=1, ms_after=2., max_spikes_per_unit=500,
    chunk_size=30000, n_jobs=6, progress_bar=True)
print(we_SC)

WaveformExtractor: 384 channels - 325 units - 1 segments
  before:30 after60 n_per_units: 500


In [10]:
comp = si.compare_two_sorters(SC, TDC, sorting1_name='SC', sorting2_name='TDC')
si.plot_agreement_matrix(comp)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## SC

In [11]:
SC.unit_ids

array([  0,   1,  10, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109,
        11, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119,  12, 120,
       121, 122, 123, 124, 125, 126, 127, 128, 129,  13, 130, 131, 132,
       133, 134, 135, 136, 137, 138, 139,  14, 140, 141, 142, 143, 144,
       145, 146, 147, 148, 149,  15, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159,  16, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169,  17, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179,  18,
       180, 181, 182, 183, 184, 185, 186, 187, 188, 189,  19, 190, 191,
       192, 193, 194, 195, 196, 197, 198, 199,   2,  20, 200, 201, 202,
       203, 204, 205, 206, 207, 208, 209,  21, 210, 211, 212, 213, 214,
       215, 216, 217, 218, 219,  22, 220, 221, 222, 223, 224, 225, 226,
       227, 228, 229,  23, 230, 231, 232, 233, 234, 235, 236, 237, 238,
       239,  24, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249,  25,
       250, 251, 252, 253, 254, 255, 256, 257, 258, 259,  26, 26

In [12]:
SC_units_ID = SC.unit_ids[25:30]
si.plot_unit_waveforms(we_SC, 
                       channel_ids=SC_units_ID,
                       max_channels=12, 
                       #radius_um=60
                      )

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

ValueError: 120 is not in list

In [14]:
si.plot_unit_templates(we_SC, 
                       channel_ids=SC_units_ID, # for some reason I can't seem to select units?
                       max_channels=12, 
                       #radius_um=60
                      )

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

ValueError: 120 is not in list

In [15]:
w_depth_SC = si.plot_units_depth_vs_amplitude(we_SC)
w_depth_SC.ax.set_aspect(0.1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/arthur/GitHub/spikeinterface/spikeinterface/core/baserecording.py:313: UserWarning: There is no Probe attached to this recording. Creating a dummy one with contact positions
  warn('There is no Probe attached to this recording. Creating a dummy one with contact positions')


## TDC

In [16]:
TDC.unit_ids

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

In [17]:
TDC_units_ID = TDC.unit_ids[:] # I couldnt use just unit 2, was giving me an into has not length error (see below)
si.plot_unit_waveforms(we_TDC, 
                       #channel_ids=TDC_units_ID
                       max_channels=12, 
                       #radius_um=60
                      )

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
si.plot_unit_templates(we_TDC, 
                       #channel_ids=TDC_units_ID,
                       max_channels=12, 
                       #radius_um=60
                      )

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
si.plot_amplitudes_distribution(we_TDC)

In [27]:
w_depth_TDC = si.plot_units_depth_vs_amplitude(we_TDC)
w_depth_TDC.ax.set_aspect(0.1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/arthur/GitHub/spikeinterface/spikeinterface/core/baserecording.py:313: UserWarning: There is no Probe attached to this recording. Creating a dummy one with contact positions
  warn('There is no Probe attached to this recording. Creating a dummy one with contact positions')
